# Simple knapsack

## Intro

This is a basic integer linear programming problem in which the goal is to maximize the value transported in a knapsack limited by capacity by selecting the most suitable items.

In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import linprog
import pyomo.environ as pyo

# Relaxed linear problem

A linear problem can be written as:

$$
\begin{align}
    \text{minimize}~ \;\; & \boldsymbol{c}^T \boldsymbol{x} \\
    \text{subject to}~ \;\; & \boldsymbol{A}_{eq} \boldsymbol{x} = \boldsymbol{b}_{eq} \\
    & \boldsymbol{A}_{ub} \boldsymbol{x} \leq \boldsymbol{b}_{ub}\\
    & \boldsymbol{l} \leq \boldsymbol{x} \leq \boldsymbol{u}
\end{align}
$$

## Formulation

**Decision variables:**

$\boldsymbol{x}$ - Column vector with the amount of each item added to the knapsack

**Fixed parameters**

$\boldsymbol{c}$ - Cost associated with each decision variable (negative of its value)

$\boldsymbol{A}_{ub}$ - Matrix of inequality constraint terms $\space\\$
$a_{1, i}$ Weight per unit of item $i\\$
$a_{2, i}$ Volume per unit of item $i$

$kw$ - Knapsack weight capacity $\space\\$
$kv$ - Knapsack volume capacity

$b_{ub} = \begin{bmatrix} kw \\ kv \end{bmatrix}$

In [2]:
#Set of items
I = set(range(1, 11))

#Random seed
np.random.seed(12)

#Weight associated with each item
w = dict(zip(I, np.random.normal(loc=5.0, scale=1.0, size=10).clip(0.5, 10.0)))

#Volume associated with each item
v = dict(zip(I, np.random.normal(loc=6.0, scale=2.0, size=10).clip(0.5, 10.0)))

#Price associated with each item
price = dict(zip(I, np.random.normal(loc=10.0, scale=1.0, size=10).clip(0.5, 20.0)))

#knapsack capacity
kw, kv = 21.0, 22.0

In [3]:
#Costs
c = -np.array(list(price.values()))

#Inequality constraints matrix
A_ub = np.array([
    np.array(list(w.values())),
    np.array(list(v.values()))
])

#Upper bounds for linear inequality constraints
b_ub = np.array([kw, kv])

#Bounds (one quantity of each item)
bounds = [(0, 1),] * 10

In [4]:
sol = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds)

In [5]:
print(sol)

           con: array([], dtype=float64)
 crossover_nit: 0
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -44.81724486288822
       ineqlin:  marginals: array([-0.92802652, -0.88281797])
  residual: array([0., 0.])
         lower:  marginals: array([0.        , 0.        , 1.4563291 , 0.        , 4.16044983,
       0.51043974, 3.41128871, 0.        , 0.        , 5.38565586])
  residual: array([1.        , 0.86557109, 0.        , 1.        , 0.        ,
       0.        , 0.        , 0.88051214, 1.        , 0.        ])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: 2
         slack: array([0., 0.])
        status: 0
       success: True
         upper:  marginals: array([-1.18142612,  0.        ,  0.        , -4.31416572,  0.        ,
        0.        ,  0.        ,  0.        , -0.41110075,  0.        ])
  residual: array([0.        , 0.13442891, 1.        , 0.        , 1.   

In [6]:
for i, item in enumerate(I):
    xi = sol.x[i]
    print(f"Item {item}: {xi:.3f}")

Item 1: 1.000
Item 2: 0.866
Item 3: 0.000
Item 4: 1.000
Item 5: 0.000
Item 6: 0.000
Item 7: 0.000
Item 8: 0.881
Item 9: 1.000
Item 10: 0.000


We have Items 2 and 8 fraction, which might be unrealistic for some situations. In those, an integer programming algorithm is necessary.

## Integrality constaints

Since version 1.9.0 scipy accepts integrality constraints as it now as a wrapper to a MILP solver.

$x_i \in \Z \space \forall i \in I$

In [8]:
integrality_vector = np.full(c.shape[0], 1)

sol_int = linprog(c, A_ub=A_ub, b_ub=b_ub, bounds=bounds, integrality=integrality_vector)

In [10]:
print(sol_int)

           con: array([], dtype=float64)
 crossover_nit: -1
         eqlin:  marginals: array([], dtype=float64)
  residual: array([], dtype=float64)
           fun: -39.88187183116921
       ineqlin:  marginals: array([0., 0.])
  residual: array([2.10553798, 1.2618095 ])
         lower:  marginals: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  residual: array([1., 0., 1., 1., 0., 0., 0., 1., 0., 0.])
       message: 'Optimization terminated successfully. (HiGHS Status 7: Optimal)'
           nit: -1
         slack: array([2.10553798, 1.2618095 ])
        status: 0
       success: True
         upper:  marginals: array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.])
  residual: array([0., 1., 0., 0., 1., 1., 1., 0., 1., 1.])
             x: array([1., 0., 1., 1., 0., 0., 0., 1., 0., 0.])


In [9]:
for i, item in enumerate(I):
    xi = sol_int.x[i]
    print(f"Item {item}: {xi:.3f}")

Item 1: 1.000
Item 2: 0.000
Item 3: 1.000
Item 4: 1.000
Item 5: 0.000
Item 6: 0.000
Item 7: 0.000
Item 8: 1.000
Item 9: 0.000
Item 10: 0.000


# Integer approach

## Create the model

In [8]:
model = pyo.ConcreteModel()

## Create the sets

- Set $I$, index $i$: Items

In [9]:
model.I = pyo.Set(initialize=I)

    (type: set).  This WILL potentially lead to nondeterministic behavior in
    Pyomo


## Parameters

$kw\\$
$kv\\$
$v_i \space \forall \space i \in I\\$
$w_i \space \forall \space i \in I\\$
$c_i \space \forall \space i \in I\\$

In [10]:
#Parameters of the knapsack
model.kw = pyo.Param(initialize=kw)
model.kv = pyo.Param(initialize=kv)

In [11]:
#Parameters of the items
model.v = pyo.Param(model.I, initialize=v)
model.w = pyo.Param(model.I, initialize=w)
model.c = pyo.Param(model.I, initialize=price)

## Create variables

$x_{i}  \space \forall \space i \in I$

In [12]:
model.x = pyo.Var(model.I, within=pyo.Integers, bounds=(0, 1))

# Capacity constraints

$\displaystyle \sum_{i \in I} x_{i} v_{i} \leq kv\\ \sum_{i \in I} x_{i} w_{i} \leq kw$

In [13]:
def weight_constraint(model):
    
    return sum(model.x[i] * model.w[i] for i in model.I) <= model.kw

model.weight_constraint = pyo.Constraint(rule=weight_constraint)

def volume_constraint(model):
    
    return sum(model.x[i] * model.v[i] for i in model.I) <= model.kv

model.volume_constraint = pyo.Constraint(rule=volume_constraint)

## Objective function

As $c_i$ is defined here as the positive corresponding value,

$\displaystyle \text{maximize} \space \sum_{i \in I} x_{i} c_{i}$

In [14]:
def obj_function(model):
    
    return sum(model.x[i] * model.c[i] for i in model.I)
    
model.objective = pyo.Objective(rule=obj_function, sense=pyo.maximize)

In [15]:
opt = pyo.SolverFactory('glpk', executable='C:\\glpk-4.65\\w64\\glpsol.exe')
opt.options['tmlim'] = 120

In [16]:
solution = opt.solve(model)
print(solution)


Problem: 
- Name: unknown
  Lower bound: 39.8818718311692
  Upper bound: 39.8818718311692
  Number of objectives: 1
  Number of constraints: 3
  Number of variables: 11
  Number of nonzeros: 21
  Sense: maximize
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 27
      Number of created subproblems: 27
  Error rc: 0
  Time: 0.06283187866210938
Solution: 
- number of solutions: 0
  number of solutions displayed: 0



In [17]:
model.objective.display()

objective : Size=1, Index=None, Active=True
    Key  : Active : Value
    None :   True : 39.88187183116921


In [18]:
model.x.display()

x : Size=10, Index=I
    Key : Lower : Value : Upper : Fixed : Stale : Domain
      1 :     0 :   1.0 :     1 : False : False : Integers
      2 :     0 :   0.0 :     1 : False : False : Integers
      3 :     0 :   1.0 :     1 : False : False : Integers
      4 :     0 :   1.0 :     1 : False : False : Integers
      5 :     0 :   0.0 :     1 : False : False : Integers
      6 :     0 :   0.0 :     1 : False : False : Integers
      7 :     0 :   0.0 :     1 : False : False : Integers
      8 :     0 :   1.0 :     1 : False : False : Integers
      9 :     0 :   0.0 :     1 : False : False : Integers
     10 :     0 :   0.0 :     1 : False : False : Integers


In [19]:
sol.x

array([9.99999999e-01, 8.65571091e-01, 7.40355900e-10, 1.00000000e+00,
       2.62434803e-10, 2.98795063e-09, 2.33299681e-10, 8.80512141e-01,
       9.99999997e-01, 2.05849974e-10])

In [20]:
for i, item in enumerate(I):
    if abs(sol.x[i] - model.x[item].value) <= 1e-3:
        if sol.x[i] >= 1e-3:
            print(f"Item {item} was added in both situations")
        else:
            print(f"Item {item} was not added in any situation")
        
    elif sol.x[i] > model.x[item].value + 1e-3:
        if sol.x[i] == 1:
            print(f"Item {item} was completely added in relaxed problem only")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem only - value {xi:.2f}")
            
    elif sol.x[i] + 1e-3 < model.x[item].value:
        if sol.x[i] <= 1e-3:
            print(f"Item {item} was added only in integer problem")
        else:
            xi = sol.x[i]
            print(f"Item {item} was partially added in relaxed problem - value {xi:.2f} - but completely added in the integer version")

Item 1 was added in both situations
Item 2 was partially added in relaxed problem only - value 0.87
Item 3 was added only in integer problem
Item 4 was added in both situations
Item 5 was not added in any situation
Item 6 was not added in any situation
Item 7 was not added in any situation
Item 8 was partially added in relaxed problem - value 0.88 - but completely added in the integer version
Item 9 was partially added in relaxed problem only - value 1.00
Item 10 was not added in any situation


In [21]:
model.weight_constraint.display()

weight_constraint : Size=1
    Key  : Lower : Body               : Upper
    None :  None : 18.894462023985934 :  21.0


In [22]:
model.volume_constraint.display()

volume_constraint : Size=1
    Key  : Lower : Body              : Upper
    None :  None : 20.73819050160827 :  22.0
